In [1]:
import numpy as np
from hmmlearn import hmm


In [2]:
hmm_states = 2

model = hmm.CategoricalHMM(n_components=hmm_states)
model.emissionprob_ = np.array([[0.1,0.9],[0.9,0.1]])
model.transmat_ = np.array([[0.9,0.1],[0.1,0.9]])
model.startprob_ = np.array([0.4,0.6])
data,states = model.sample(n_samples = 1000,random_state=28)
data = np.squeeze(data)

In [3]:
# model2 = hmm.CategoricalHMM(n_components=hmm_states,n_iter=1000,random_state=30,implementation='scaling')
# model2.fit(data.reshape(-1,1))
# hidden_states = model2.predict(data.reshape(-1,1))
# # print("Most likely hidden states:", hidden_states)


In [14]:
n_states = 2
n_features = 2
n_obs = data.shape[0]
# emission = np.random.rand(n_obs,n_states)
# emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))

emission = np.random.rand(n_states,n_features)
emission = emission/np.tile(np.expand_dims(np.sum(emission,axis=1),axis=1),(1,2))
emission = np.array([[0.2,0.8],[0.3,0.7]])

transition =  np.random.rand(n_states,n_states)
transition = transition/np.tile(np.expand_dims(np.sum(transition,axis=1),axis=1),(1,2))
transition = np.array([[0.6,0.4],[0.3,0.7]])


scale_factors = np.zeros((n_obs))
forward_hat = np.zeros((n_obs,n_states))

backward_hat = np.zeros((n_obs,n_states))

init_prob = np.array([0.5,0.5])


p_old = -10000
tol = 0.01
max_iter = 100

mu = np.random.rand(n_states)

In [15]:
print(f'emission is {emission} and transition is {transition}')

emission is [[0.2 0.8]
 [0.3 0.7]] and transition is [[0.6 0.4]
 [0.3 0.7]]


In [17]:
for ite in range(max_iter):
    forward_hat = np.zeros((n_obs,n_states))
    backward_hat = np.zeros((n_obs,n_states))
    scale_factors = np.zeros((n_obs))


    forward_hat[0,:] = init_prob * emission[:,data[0]]
    scale_factors[0] = np.sum(forward_hat[0,:])
    forward_hat[0,:] = forward_hat[0,:]/scale_factors[0]
    
    for t in range(n_obs-1):
        temp = np.matmul(forward_hat[t,:] ,transition) * emission[:,data[t+1]]
        scale_factors[t+1] = np.sum(temp)
        forward_hat[t+1,:] = temp/scale_factors[t+1]
        # print(f'temp is {temp} and the scale factor is {scale_factors[t+1]} and the forward_hat is {forward_hat[t+1]}')


    backward_hat[-1,:] = scale_factors[-1]
    for t in reversed(range(n_obs-1)):
        temp = np.matmul(backward_hat[t+1,:]*emission[:,data[t+1]],transition.T)
        backward_hat[t,:] = temp/scale_factors[t]
        


    a = np.zeros((n_obs,n_states))
    b = np.zeros((n_obs,n_states,n_states))
    for i in range(n_obs):
        for j in range(n_states):
            a[i,j] = forward_hat[i,j]*backward_hat[i,j]
        temp = np.sum(a[i,:])
        a[i,:] = a[i,:]/temp

    for t in range(n_obs-1):
        for i in range(n_states):
            for j in range(n_states):
                b[t,i,j] = scale_factors[t+1]*forward_hat[t,i]*backward_hat[t+1,j]*transition[i,j]*emission[j,data[t+1]]

    for i in range(n_states):
        for j in range(n_states):
            transition[i,j] = np.sum(b[0:-1,i,j])/np.sum(b[0:-1,i,:])
            # print(np.sum(b[0:-1,i,j]),np.sum(b[0:-1,i,:]))

    for i in range(n_states):
        init_prob[i] = a[0,i]/np.sum(a[0,:])

    for i in range(n_states):
        for j in range(n_states):
            emission[j,i] = np.sum(a[np.argwhere(data==i),j]) / np.sum(a[:,j])
            
    p = np.sum(np.log(scale_factors))
    print(f'p is:{p},transition is {transition},emission is {emission},init is {init_prob}')
    # print(f'transition prob is: {transition}')
    # print(f'emission prob is:{emission}')
    if p>p_old and p - p_old < tol:
        break
    p_old = p

[3.94266517e-33 1.00000000e+00]
[1.63334405e-04 9.99836666e-01]
[6.08751721e-04 9.99391248e-01]
[0.00409224 0.99590776]
[0.03216757 0.96783243]
[0.03414526 0.96585474]
[0.01518214 0.98481786]
[0.01651864 0.98348136]
[0.00235204 0.99764796]
[6.84842485e-04 9.99315158e-01]
[0.00120137 0.99879863]
[0.00709757 0.99290243]
[0.05485572 0.94514428]
[0.06144614 0.93855386]
[0.14294046 0.85705954]
[0.8035782 0.1964218]
[0.95443576 0.04556424]
[0.98887181 0.01112819]
[0.99667777 0.00332223]
[0.99820758 0.00179242]
[0.99745016 0.00254984]
[0.99243056 0.00756944]
[0.97006029 0.02993971]
[0.97599852 0.02400148]
[0.96965526 0.03034474]
[0.93449058 0.06550942]
[0.77881312 0.22118688]
[0.09669641 0.90330359]
[0.01210226 0.98789774]
[0.00161117 0.99838883]
[3.10409704e-04 9.99689590e-01]
[1.51583491e-04 9.99848417e-01]
[1.51971806e-04 9.99848028e-01]
[3.13977311e-04 9.99686023e-01]
[0.00164 0.99836]
[0.01233466 0.98766534]
[0.09857037 0.90142963]
[0.79392364 0.20607636]
[0.95271171 0.04728829]
[0.98897